## Tarea 2

### Integrantes:

+ Iván Alvarez Tostado Bárcena
+
+

### Indicaciones

Implementar los procedimientos:
+ *Validación cruzada*
+ *Perceptrón*
+ *Descenso por gradiente en MVS*
+ *Descenso por gradiente estocástico*

Probar CV con perceptrón y descenso por gradiente estocástico

In [1]:
from sklearn import datasets
import pandas as pd
import numpy as np
import random
import math

In [2]:
random.seed(123)

In [3]:
iris = datasets.load_iris()

In [4]:
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                  columns= iris['feature_names'] + ['Species'])


In [5]:
tgt = iris.target[0]

In [6]:
tgt_name = iris.target_names[0]
tgt_name

'setosa'

En el feature 'Species' vemos que 'setosa' corresponde al valor '0', por lo que asignaremos el valor 1 si se trata de 'setosa' y -1 en el caso de que se trate de 'versicolor' o 'virginica'.

In [7]:
df['Species'] = df['Species'].replace([0,1,2],[1,-1,-1])
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Species
0,5.1,3.5,1.4,0.2,1.0
1,4.9,3.0,1.4,0.2,1.0
2,4.7,3.2,1.3,0.2,1.0
3,4.6,3.1,1.5,0.2,1.0
4,5.0,3.6,1.4,0.2,1.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,-1.0
146,6.3,2.5,5.0,1.9,-1.0
147,6.5,3.0,5.2,2.0,-1.0
148,6.2,3.4,5.4,2.3,-1.0


Comenzamos con la función **cross_validate_splits**, la cual tiene como parámetros un dataframe y un número *k* que indica la cantidad de particiones (folds) en las que dividiremos el conjunto de datos. 

Esta función toma el df original y reparte, de forma aleatoria, los elementos en arreglos del tamaño n/*k*.

In [8]:
def cross_validate_splits(df, folds):
    cv_splits = []
    index = df.index.values.tolist()
    lenght_split = int(len(index)/folds)
    
    #Divide data into random folds
    for x in range(folds):
        
        split = []
        while  lenght_split > len(split):
            
            #select a random element index
            element_index =  random.choice(index)  
            #append row to split list
            split.append(df[df.index == element_index].values[0].tolist())
            #ensure no duplicates 
            index.remove(element_index)    
        
        #append each fold to cv_splits list
        cv_splits.append(np.asarray(split))

        
    return cv_splits



Esta siguiente función, **cross_validate_lists**, toma los mismos parámetros que la función anterior, y se encarga de crear tuplas con (como se encuentra en la siguiente imagen):
+ Datos de entrenamiento (k-1)
+ Datos de prueba (1)

<img src=https://scikit-learn.org/stable/_images/grid_search_cross_validation.png>

In [9]:
def cross_validate_lists(df, folds):
    
    #initial values
    #separation of df into splits
    cv_splits = cross_validate_splits(df, folds)
    #list of D without Di
    cv_folds_train = list()
    cv_folds_test = list()
    
    #Check that each fold does not have Di
    for i in range(len(cv_splits)):
        df_train = list()
        df_test = list()
        for j in range(len(cv_splits)):
            if i != j:
                df_train.append(cv_splits[j])
            else:
                df_test.append(cv_splits[j])
                
        
        df_train = np.vstack(df_train)
        df_test = np.vstack(df_test)
        cv_folds_train.append(df_train)
        cv_folds_test.append(df_test)
        
    return cv_folds_train, cv_folds_test

La función **perceptrón** utiliza un conjunto de datos y regresa una theta y theta_0 estimadas.

In [10]:
def perceptron(T, Dn):
    #features without clasification
    df_lenght = Dn.shape[1]-1
    
    #initial values
    theta_0 = 0
    theta = np.zeros(df_lenght).transpose()
    
    #total of runs
    for t in range(T):
        #total of rows in Dn
        for i in range(Dn.shape[0]):
            #Check and update initial values
            if Dn[i][df_lenght]*(np.dot(theta, Dn[i][:df_lenght,]) + theta_0) <= 0:
                theta += Dn[i][df_lenght] * Dn[i][:df_lenght,]
                theta_0 +=  Dn[i][df_lenght]

    return theta, theta_0
    

La validación cruzada aplica las funciones de gradiente por descenso estocástico/perceptrón y obtiene las thetas con los folds de entrenamiento. Luego, con los valores arrojados entrena los datos de prueba y regresa el promedio de los errores obtenidos para los k folds.

In [11]:
def cross_validate(df, folds, function = 'perceptron'):
    df_train = cross_validate_lists(df,folds)[0]
    df_test = cross_validate_lists(df,folds)[1]
    df_lenght = df_train[0].shape[1]-1

    
    errors = list()
    parameters = list()
    
    
    if function == 'perceptron':
        for i in range(len(df_train)):
            #Toma el set de entrenamiento y aplica la función perceptrón
            parameters.append(perceptron(10,df_train[i]))
            #Con los valores arrojados de theta y theta_0 entrena los datos
            for j in range(len(df_test[i])):
                error = 0
                theta = parameters[i][0]
                theta_0 = parameters[i][1]
                 #Suma los errores si la predicción no cumple con el caso real
                if df_test[i][j][df_lenght]*(np.dot(theta, df_test[i][j][:df_lenght,]) + theta_0) <= 0:
                    error += 1
            errors.append(error)
        
            
        
    
    return np.average(errors)


In [12]:
#Prueba de CV con función perceptrón
cross_validate(df,5)


0.0